In [16]:
import tensorflow as tf
with tf.Session() as sess:
  devices = sess.list_devices()
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))



In [32]:
DESIRED_ACCURACY = 0.95
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>DESIRED_ACCURACY):
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

In [27]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax')
])

In [28]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 9248)             

In [37]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [38]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255,
                                   rotation_range=0,
                                   width_shift_range=0.4,
                                   height_shift_range=0.4,
                                   brightness_range=[0.4, 1.5],
                                   shear_range=0.2,
                                   zoom_range=0.5,
                                   fill_mode='nearest',)

train_generator = train_datagen.flow_from_directory(
        "signs_data/",  
        target_size=(150, 150), 
        batch_size=10,
        class_mode='sparse')

Found 2278 images belonging to 25 classes.


In [39]:
model.fit_generator(train_generator, epochs=100, verbose=1, callbacks=[callbacks])

Epoch 1/100
228/228 [==============================] - 23s 103ms/step - loss: 2.5394 - acc: 0.2559
Epoch 2/100
208/228 [==========================>...] - ETA: 2s - loss: 2.3046 - acc: 0.3065

KeyboardInterrupt: 